In [1]:
%load_ext autoreload
%autoreload 2

from scripts import hdc_scraping
from IPython.display import clear_output

In [12]:
# Define what you need to extract
indicator = 'Home visit palliative care'
url = 'https://hdcservice.moph.go.th/hdc/reports/report.php?&cat_id=b08560518ca0ebcaf2016dab69fb38b5&id=8eb4ec8a40c080aadfdbb895be74e771'
level = 'รายอำเภอ' # Select from ['เขตพื้นที่', 'Service Plan', 'รายโรงพยาบาล', 'รายหน่วยบริการ', 'เครือข่ายบริการ']
yearback = 1

In [13]:
hdc_scraping.scraping(indicator, url, level, yearback)

clear_output()
print("Scraped successfully")

Extracting Home visit palliative care: https://hdcservice.moph.go.th/hdc/reports/report.php?&cat_id=b08560518ca0ebcaf2016dab69fb38b5&id=8eb4ec8a40c080aadfdbb895be74e771
เขตสุขภาพที่ 1
done
เขตสุขภาพที่ 2
done
เขตสุขภาพที่ 3


KeyboardInterrupt: 

In [50]:
indicators_df = pd.read_excel('data/indicator_list.xlsx')
indicators_df['indicator'] = indicators_df['indicator'].apply(lambda x: f'indicator_{x:02}')
indicators_df = indicators_df[indicators_df['indicator'].isin(['indicator_27', 'indicator_28', 'indicator_29'])]
display(indicators_df.head())

,indicator,name,name_source,website,url,numerator,denomenator,reversed,source,header,col_A,col_B,col_A/B,col_area
26,indicator_27,ร้อยละของการให้การรักษาผู้ป่วย STEMI ไม่ได้ตาม...,NaN,NaN,https://hdcservice.moph.go.th/hdc/reports/repo...,NaN,NaN,1.0,NaN,3.0,6.0,5.0,7.0,1.0
27,indicator_28,อัตราตายทารกไทยแรกเกิด อายุน้อยกว่าหรือเท่ากับ...,NaN,NaN,https://hdcservice.moph.go.th/hdc/reports/repo...,NaN,NaN,0.0,NaN,1.0,3.0,2.0,4.0,1.0
28,indicator_29,ร้อยละของผู้ป่วยประคับประคองที่ไม่ได้รับการดูแ...,NaN,NaN,https://hdcservice.moph.go.th/hdc/reports/repo...,NaN,NaN,1.0,NaN,3.0,3.0,2.0,4.0,1.0


In [51]:
urls = indicators_df[['indicator', 'url']].dropna().set_index('indicator')['url'].to_dict()
display(urls)

{'indicator_27': 'https://hdcservice.moph.go.th/hdc/reports/report.php?cat_id=39fd60c25235db479930db85a0e97dd3&id=12d5b6eef67669da17758ef281915cbb',
 'indicator_28': 'https://hdcservice.moph.go.th/hdc/reports/report.php?&cat_id=58b45d24cc1ba2a288d0f9bb1f7d3f71&id=50617527edfd34fe8e3fc9fabac0429a',
 'indicator_29': 'https://hdcservice.moph.go.th/hdc/reports/report.php?&cat_id=b08560518ca0ebcaf2016dab69fb38b5&id=8eb4ec8a40c080aadfdbb895be74e771'}

In [52]:
# Set download path to current folder
options = Options()
base_download_dir = os.path.abspath("data/indicators")
options.add_experimental_option('prefs', {
    'download.default_directory': base_download_dir
})

# Open Edge webdriver
# User have to download Edge webdriver first from https://developer.microsoft.com/en-us/microsoft-edge/tools/webdriver/ (Stable channel -> x64)
driver = webdriver.Chrome(options=options)

In [53]:
for indicator, url in urls.items():
    
    indicator_folder = os.path.join(base_download_dir, indicator)
    os.makedirs(indicator_folder, exist_ok=True)
    
    print(f"Fetching {indicator}: {url}")  # Optional logging
    driver.get(url)
    
    # change to old interface 
    old_report_button = driver.find_element(By.XPATH, "//span[@onclick='seeReportOld()']")
    old_report_button.click()
    
    # Wait for the table to load (about 20 seconds)
    wait = WebDriverWait(driver, timeout=60)
    wait.until(lambda d: driver.find_element(By.ID, 'ptable').is_displayed())

    # ------------- 5 year download automation ---------- #
    type = Select(driver.find_element(By.ID, 'selSP'))
    type.select_by_visible_text('เขตพื้นที่')
    
    sel_year = Select(driver.find_element(By.ID, 'selYear'))
    
    # Select the last 5 years from the dropdown
    for y_option in sel_year.options[:5]: # change number to download from last N year
        year = y_option.text
        print(f'year: {year}')
        sel_year.select_by_visible_text(year) 
        
        year_folder = os.path.join(indicator_folder, year)
        os.makedirs(year_folder, exist_ok=True)

        try:
            sel_month = Select(driver.find_element(By.ID, 'selMonth'))
            sel_month.select_by_visible_text('ธันวาคม')  # <- Change report month here
        except:
            print("Dropdown not found, continuing script...")

        #  รายอำเภอ 
        sel_month = Select(driver.find_element(By.ID, 'selAllProv'))
        sel_month.select_by_visible_text('รายอำเภอ') 

        # เขต
        sel_zone = Select(driver.find_element(By.ID, 'selZone'))
        for z_option in sel_zone.options[1:]:  # Skip the first "------ ทั้งหมด ------" option
            zone_name = z_option.text
            # Select zone
            sel_zone.select_by_visible_text(z_option.text)
            
            attempts = 0
            success = False
                
            while attempts < 3 and not success:  # Retry 3 times
                try:
                    # Click ok button 
                    driver.find_element(By.ID, 'btnOkey').click()
                    wait.until(lambda d: d.find_element(By.ID, 'ptable').is_displayed())
                    
                    # Download the data
                    driver.execute_script(f"$('#dataTable').tableExport({{type:'csv',escape:false,tableName:'{indicator}_{z_option.text}_{year}'}});")
                    time.sleep(3)
                    shutil.move(os.path.join('data/indicators', f'{indicator}_{z_option.text}_{year}.csv'), os.path.join(year_folder, f'{indicator}_{z_option.text}_{year}.csv'))
                    print('done')
                    success = True  # exit the retry loop
                except Exception as e:
                    print(f"Error: {e}")
                    print("Retrying")
                    attempts += 1  
                    if attempts == 3:  # If still fails, proceed to next province
                        print("Max retries")
                        success = True
                except Exception as e:
                    print(f"Error: {e}")
                    print(f"Retrying")
                    attempts += 1
                    if attempts == 3:  # If still fails, proceed to next province
                        success = True
                        
    print(f'Finising scraped for {indicator} ==========')

Fetching indicator_27: https://hdcservice.moph.go.th/hdc/reports/report.php?cat_id=39fd60c25235db479930db85a0e97dd3&id=12d5b6eef67669da17758ef281915cbb
year: 2568
Dropdown not found, continuing script...
done
done
done
done
done
done
done
done
done
done
done
done
done
year: 2567
Dropdown not found, continuing script...
Error: Message: 
Stacktrace:
0   chromedriver                        0x00000001013330d4 cxxbridge1$str$ptr + 2600792
1   chromedriver                        0x000000010132b9f0 cxxbridge1$str$ptr + 2570356
2   chromedriver                        0x0000000100ecc3d8 cxxbridge1$string$len + 89376
3   chromedriver                        0x0000000100f11414 cxxbridge1$string$len + 372060
4   chromedriver                        0x0000000100f4b638 cxxbridge1$string$len + 610176
5   chromedriver                        0x0000000100f0534c cxxbridge1$string$len + 322708
6   chromedriver                        0x0000000100f05f94 cxxbridge1$string$len + 325852
7   chromedriver         